In [2]:
from image import *

In [3]:
# RGB image
preprocess = {"nothing": lambda x: x, "contrast": increase_contrast, "normalize": normalize, "blackhat": blackhat}
hairs = {"nothing": lambda x: x, "median": median, "low_pass_filter": low_pass_filter_rgb, "dilate": dilate}
segments = {"threshold": threshold, "watershed": watershed, "mser": mser, "k_means": k_means}

img_folder = "images/original"
seg_folder = "images/segmented"

results = {}

for pre in preprocess:
    for hair in hairs:
        for seg in segments:

            name = "{} - {} - {}".format(pre, hair, seg)
            results[name] = {}
            avg_TPR = 0
            avg_FPR = 0
            avg_TP = 0
            avg_TN = 0
            avg_FP = 0
            avg_FN = 0
            cont = 0

            for img_name in sorted(os.listdir(img_folder)):
                cont += 1
                aux_index = img_name.index("i")
                n = img_name[0:aux_index]
                # Real image
                img_path = img_folder + "/" + img_name
                img = load_image_rgb(img_path)

                # Segmented by function
                func_seg_img = segments[seg](hairs[hair](preprocess[pre](img)))

                # Improve the segmentation by applying morphology operations
                opening = cv2.morphologyEx(func_seg_img, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))
                final = cv2.morphologyEx(opening, cv2.MORPH_DILATE, np.ones((5,5),np.uint8),iterations=3)
                final = cv2.medianBlur(final,9)
                final = cv2.morphologyEx(final, cv2.MORPH_DILATE, np.ones((7,7),np.uint8),iterations=3)
                final = cv2.medianBlur(final,11)
                func_seg_img = cv2.morphologyEx(final, cv2.MORPH_CLOSE, np.ones((7,7),np.uint8))

                # Real segmentation
                real_seg_path = seg_folder + "/" + n + "seg.png"
                real_seg_img = load_image(real_seg_path)

                # Compare
                TPR, FPR, TP, TN, FP, FN = compare_seg(real_seg_img, func_seg_img)
                this_res = {
                    "TP": TP,
                    "FP": FP,
                    "TN": TN,
                    "FN": FN,
                    "TPR": TPR,
                    "FPR": FPR,
                }
                results[name][n] = this_res
                avg_TPR += TPR
                avg_FPR += FPR
                avg_TP += TP
                avg_TN += TN
                avg_FP += FP
                avg_FN += FN

            results[name]["average_TPR"] = avg_TPR/cont
            results[name]["average_FPR"] = avg_FPR/cont
            results[name]["average_TP"] = avg_TP/cont
            results[name]["average_TN"] = avg_TN/cont
            results[name]["average_FP"] = avg_FP/cont
            results[name]["average_FN"] = avg_FN/cont
            
            # print(name)
            # print("Avg TPR:", results[name]["average_TPR"])
            # print("Avg FPR:", results[name]["average_FPR"])
            # print()       



nothing - nothing - threshold
Avg TPR: 0.9467833069495523
Avg FPR: 0.2044462853793481

nothing - nothing - k_means
Avg TPR: 0.964958948601586
Avg FPR: 0.20405029281444675

nothing - median - threshold
Avg TPR: 0.9215622851343842
Avg FPR: 0.14576848497391112

nothing - median - k_means
Avg TPR: 0.9455422102197866
Avg FPR: 0.14084598250050928

nothing - low_pass_filter - threshold
Avg TPR: 0.9271735265706357
Avg FPR: 0.15279052467023793

nothing - low_pass_filter - k_means
Avg TPR: 0.949680172577408
Avg FPR: 0.15109830858135895

nothing - dilate - threshold
Avg TPR: 0.939680352068171
Avg FPR: 0.17480645401408246

nothing - dilate - k_means
Avg TPR: 0.9548172421680493
Avg FPR: 0.16906597038878954

contrast - nothing - threshold
Avg TPR: 0.954510147999308
Avg FPR: 0.20340868055542316

contrast - nothing - k_means
Avg TPR: 0.969215488354556
Avg FPR: 0.2093578247202849

contrast - median - threshold
Avg TPR: 0.9298223469174982
Avg FPR: 0.14385752929060852

contrast - median - k_means
Avg TPR

In [1]:
# Random metric 
def metric2(TPR,FPR):
    return (1+(1-TPR))**2 + (1+FPR)**2 - 2

def metric1(TPR,FPR):
    return 1-TPR+FPR

for pre in preprocess:
    for hair in hairs:
        for seg in segments:
            name = "{} - {} - {}".format(pre, hair, seg)
            TPR = results[name]["average_TPR"]
            FPR = results[name]["average_FPR"]
            simple_dif = metric1(TPR,FPR)
            squared_dif = metric2(TPR,FPR)
            if simple_dif < 0.2:
                print(name)
                print("Avg TPR:", TPR)
                print("Avg FPR:", FPR)
                print("Simple diff:", simple_dif)
                print("Squared diff:", squared_dif)
                print()   


NameError: name 'preprocess' is not defined